In [5]:
import gc, os
import pickle
import cfgrib
import pygrib

import pandas as pd
import numpy as np
import xarray as xr
import multiprocessing as mp
import matplotlib.pyplot as plt

from glob import glob
from datetime import datetime, timedelta
from functools import partial, reduce

os.environ['OMP_NUM_THREADS'] = '1'

In [6]:
svr_model_config = 'clnx'
site, interval, delay = 'CLNX', 12, 12
site_lat, site_lon = 40.5763, -111.6383

In [7]:
model = 'gfs0p25'

temp = '/scratch/general/lustre/u1070830/binary_temp/'; os.makedirs(temp, exist_ok=True)
archive = '/uufs/chpc.utah.edu/common/home/steenburgh-group10/mewessler/archive/'

obs_dir = '/uufs/chpc.utah.edu/common/home/steenburgh-group10/mewessler/observations/clean/'
mlmodel_dir = '/uufs/chpc.utah.edu/common/home/steenburgh-group10/mewessler/output/slr_models/%s/'%svr_model_config

date_fmt = '%Y%m%d'
datetime_fmt = '%Y%m%d%H'

In [9]:
df = pd.read_pickle('./%s_%dh_delay%d_extract_gfs_ISO.pd'%(site, interval, delay))
df

,T_400,T_450,T_500,T_550,T_600,T_650,T_700,T_750,T_800,T_850,...,V100M,MSL,CAPE,SP,swe_mm,BLH,DIR10M,SPD10M,DIR100M,SPD100M
time,,,,,,,,,,,,,,,,,,,,,
2015-01-15 15:00:00,242.899994,250.100006,256.500000,262.200012,267.000000,270.600006,272.500000,271.799988,269.600006,272.700012,...,0.510000,103223.421875,0.0,79931.679688,0.0000,21.040001,101.768288,1.716071,105.736137,1.880479
2015-01-15 18:00:00,242.899994,250.199997,256.700012,262.399994,267.299988,271.100006,273.299988,273.299988,272.000000,275.200012,...,0.930000,103163.046875,0.0,79990.484375,0.0000,62.000000,108.043411,1.388272,118.681252,1.937756
2015-01-15 21:00:00,243.100006,250.399994,256.700012,262.299988,267.200012,271.200012,273.399994,273.799988,274.700012,277.899994,...,1.350000,102743.140625,0.0,79813.304688,0.0000,54.000000,153.434952,0.939149,148.109161,1.590001
2015-01-16 00:00:00,243.100006,250.000000,256.299988,261.899994,266.700012,270.600006,273.000000,274.299988,274.700012,277.799988,...,2.430000,102764.148438,0.0,79811.828125,0.0000,22.000000,156.849121,1.576991,161.777527,2.558300
2015-01-16 03:00:00,242.699997,249.800003,256.000000,261.700012,266.600006,270.500000,272.700012,273.799988,272.200012,275.399994,...,1.750000,102909.421875,0.0,79816.960938,0.0000,21.040001,109.093491,1.925954,136.684677,2.405203
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-06-01 00:00:00,246.399979,253.100006,259.399994,264.320862,269.152313,274.600006,280.299988,285.799988,290.799988,294.299988,...,-0.395981,101238.343750,333.0,79600.398438,0.0000,1079.398682,288.076904,1.497584,284.714294,1.558987
2019-06-01 03:00:00,247.099976,254.000000,258.799988,263.600006,269.086914,274.499969,279.000000,283.399994,287.500000,290.600006,...,4.215518,101322.609375,103.0,79649.351562,0.2500,73.487442,193.631805,2.639875,191.192581,4.297250
2019-06-01 06:00:00,246.199982,252.799973,258.799988,264.399994,269.850098,275.299988,280.100006,283.899994,285.399994,288.299988,...,0.296567,101472.976562,50.0,79667.023438,0.8125,31.354668,92.758636,1.668274,96.626389,2.570027


In [ ]:
scaler_file = glob(mlmodel_dir + '*scaler*')[-1]
stats_file = glob(mlmodel_dir + '*train_stats*')[-1]
model_file = glob(mlmodel_dir + '*SLRmodel*')[-1]

with open(scaler_file, 'rb') as rfp:
    scaler = pickle.load(rfp)

with open(stats_file, 'rb') as rfp:
    train_stats, train_stats_norm = pickle.load(rfp)
    model_keys = train_stats.keys()

with open(model_file, 'rb') as rfp:
    SLRmodel = pickle.load(rfp)

In [ ]:
model_keys

In [ ]:
df_trim = df.loc[:, model_keys]
df_norm = pd.DataFrame(scaler.transform(df_trim), index=df_trim.index, columns=df_trim.keys())
df_norm

In [ ]:
slr = pd.DataFrame(SLRmodel.predict(df_norm), 
                   index=df_norm.index, columns=['slr'])

slr[slr < 0] = 0
output = slr.merge(df['swe_mm'], on='time')
output['snow_mm'] = output['swe_mm'] * output['slr']

output = output.resample('12H', convention='end', label='right', closed='right').sum()[['swe_mm', 'snow_mm']]
output['slr_weighted'] = output['snow_mm']/output['swe_mm']
output.index = output.index.rename('valid')
output

In [ ]:
obs_file = glob(obs_dir + '%s*.pd'%site)[0]

obs = pd.read_pickle(obs_file).reset_index().rename(
    columns={'datetime_utc':'valid'})#.set_index('valid')

# CONSIDER FIXING OFFSET IN INGEST (CLN SHOULD BE 04/16 MST 11/23 UTC)
obs['valid'] = obs['valid'] + timedelta(hours=3)
obs = obs.set_index('valid')
    
obs = obs[[k for k in obs.keys() if str(interval) in k]]
obs = obs.rename(columns={k:'obs_' + k.replace('%s'%interval, '') for k in obs.keys()})

swe_min = 2.54 #0.1 in 
obs = obs[obs['obs_swe_mm'] >= 2.54]

obs

In [ ]:
match = obs.merge(output, on='valid')
match = match.dropna(how='any')
match

In [ ]:
plt.scatter(match['obs_slr'], match['slr_weighted'])